In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error




In [2]:
# Load dataset (CIFAR-10 as an example)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

In [3]:
# Define Explicit Input Tensor (Fixes Model Input Error)
input_tensor = keras.Input(shape=(32, 32, 3))

x = layers.Conv2D(32, (3, 3), activation='relu')(input_tensor)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)  # Feature Extraction before Dense
x = layers.Dense(128, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)



In [4]:

model = models.Model(inputs=input_tensor, outputs=output_tensor)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [5]:
# Train the CNN Model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.3836 - loss: 1.6663 - val_accuracy: 0.6015 - val_loss: 1.1292
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.6222 - loss: 1.0683 - val_accuracy: 0.6549 - val_loss: 0.9943
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.6926 - loss: 0.8861 - val_accuracy: 0.6892 - val_loss: 0.8946
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.7317 - loss: 0.7634 - val_accuracy: 0.7035 - val_loss: 0.8579
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.7666 - loss: 0.6639 - val_accuracy: 0.7128 - val_loss: 0.8516


In [6]:
# Identify the last convolutional layer dynamically
conv_layer_index = max(i for i, layer in enumerate(model.layers) if isinstance(layer, layers.Conv2D))

In [7]:
# Extract feature maps from the last convolutional layer
feature_extractor = models.Model(inputs=model.input, outputs=model.layers[conv_layer_index].output)

In [8]:
# Extract Features
features_train = feature_extractor.predict(x_train, verbose=1)
features_test = feature_extractor.predict(x_test, verbose=1)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [9]:
# Flatten features for ML classifiers
features_train = features_train.reshape(features_train.shape[0], -1)
features_test = features_test.reshape(features_test.shape[0], -1)

In [10]:

# Standardize Features
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

In [11]:
# Train ML Classifiers
classifiers = {
    "kNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

for name, clf in classifiers.items():
    clf.fit(features_train, y_train.ravel())  # Train classifier
    y_pred = clf.predict(features_test)  # Predict on test features
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")

kNN Accuracy: 0.6232
Decision Tree Accuracy: 0.4131
Random Forest Accuracy: 0.6614


In [ ]:
regressor_results = {}

regressors = {
    "Linear Regression": LinearRegression(),
    "LASSO": Lasso(alpha=0.1),
    "k-NN Regressor": KNeighborsRegressor(n_neighbors=5),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100),
    "ADA Boosted Trees": AdaBoostRegressor(n_estimators=50),
    "Gradient Boosted Trees": GradientBoostingRegressor(n_estimators=100),
    "XGBoost Regressor": XGBRegressor(n_estimators=100, objective='reg:squarederror')
}

print("\n=== Regression Results (Mean Squared Error) ===")
for name, reg in regressors.items():
    reg.fit(features_train, y_train.ravel())  # Train regressor
    y_pred = reg.predict(features_test)  # Predict on test features
    mse = mean_squared_error(y_test, y_pred)
    regressor_results[name] = mse
    print(f"{name} MSE: {mse:.4f}")



=== Regression Results (Mean Squared Error) ===
Linear Regression MSE: 6.2945
LASSO MSE: 5.7091
k-NN Regressor MSE: 4.6850
Decision Tree Regressor MSE: 10.7634
